In [2]:
from dataset import load_image
from tqdm import tqdm

In [6]:
data_path = '/home/workspace/iot_ai_model/dataset/supervisely_person/test_data_list.txt'
with open(data_path, 'r') as f:
    line = f.readlines()

base = '/home/workspace/iot_ai_model/dataset/supervisely_person/'

for p in tqdm(line):
    filename = p.split(',')
    load_image(base + filename[0])

In [7]:
with open(data_path, 'r') as f:
    line = f.readlines()

In [8]:
line[0]

'images/ds5_pexels-photo-245241.png,masks/ds5_pexels-photo-245241.png\n'

In [9]:
base = '/home/workspace/iot_ai_model/dataset/supervisely_person/'

In [10]:
for p in tqdm(line):
    filename = p.split(',')
    load_image(base + filename[0])

100%|██████████| 534/534 [00:03<00:00, 162.41it/s]
